* Exercise - Setup CICD for Streamlit Application
* Setup GitHub repository for the Application
* Manual Deployment on GCP VM
* Setup Supervisor and Nginx
* Add Workflow for GitHub Action
* Validate Deployed Streamlit Application

* Setup GitHub repository for the Application

Make sure to have `.gitignore` with the files and folders that are supposed to be ignored. Also, add `requirements.txt` with `streamlit` along with the version.

```shell
git init
git add .
git commit -m "Initial Commit"

# Setup GitHub repo and run commands to keep local and GitHub repos in synch.
```

* Manual Deployment on GCP VM

Login to the VM and run the below commands to take care of the manual deployment. Make sure to validate to see if the application is accessible or not.

```shell
git clone <git url for streamlit app>
sudo mkdir -p /var/log/streamlit-app
cd streamlit-app

python -m venv sa-venv
source sa-venv/bin/activate
pip install -r requirements.txt

sudo systemctl stop nginx
sudo streamlit run app.py --server.port 80
```

* Setup Supervisor and Nginx

Here is the `streamlit-app.conf` content which should be placed under `/etc/supervisor/conf.d`.

```text
[program:streamlit-app]
directory=/home/itversity/streamlit-app
command=/home/itversity/streamlit-app/sa-venv/bin/streamlit run /home/itversity/streamlit-app/app.py --server.headless true
autostart=true
autorestart=true
stderr_logfile=/var/log/streamlit-app/streamlit-app.err.log
stdout_logfile=/var/log/streamlit-app/streamlit-app.out.log
```

Here is the updated content of `/etc/nginx/nginx.conf`

```text
events {
    worker_connections 1024; # increase if you have lots of clients
    accept_mutex off; # set to 'on' if nginx worker_processes > 1
}

http {
    server {
        listen 80;
        server_name _;

        location / {
            proxy_pass http://0.0.0.0:8501/;
            proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
            proxy_set_header Host $http_host;
            proxy_redirect off;
            proxy_http_version 1.1;
            proxy_set_header Upgrade $http_upgrade;
            proxy_set_header Connection "upgrade";
        }
    }
}
```

Run below commands to restart nginx and supervisor.

```shell
sudo systemctl restart nginx
sudo systemctl restart supervisor
sudo supervisorctl status
```

* Add Workflow for GitHub Action

1. Make sure to add secrets.
2. Add `setup.sh` in the base folder.

```shell
python3 -m venv /home/itversity/streamlit-app/sa-venv
source /home/itversity/streamlit-app/sa-venv/bin/activate
pip install -r /home/itversity/streamlit-app/requirements.txt
sudo supervisorctl restart streamlit-app
```

3. Add a file by name `streamlit-app.yml` under `.github/workflows` in base directory of application folder.

```yaml
name: Streamlit App

on:
  push:
    branches: [ "main" ]
  pull_request:
    branches: [ "main" ]

permissions:
  contents: read

jobs:
  build:

    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v3
    - name: Enable Passwordless Login
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
        APP_SSH_SECRET_KEY: ${{ secrets.APP_SSH_SECRET_KEY }}
      run: |
        mkdir -p /tmp/keys && chmod 700 /tmp/keys
        echo "$APP_SSH_SECRET_KEY" > /tmp/keys/private_key && chmod 600 /tmp/keys/private_key
    - name: Create Folder and Deploy Application
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
      run: |
        ssh -o StrictHostKeyChecking=no -i /tmp/keys/private_key ${APP_SSH_USER}@${APP_HOST} '
          mkdir -p ~/streamlit-app
        '
        scp -i /tmp/keys/private_key -r * ${APP_SSH_USER}@${APP_HOST}:~/streamlit-app
    - name: Build and Run Application
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
      run: |
        ssh -i /tmp/keys/private_key ${APP_SSH_USER}@${APP_HOST} "/bin/bash -c 'source ~/streamlit-app/setup.sh'"
```

* Validate Deployed Streamlit Application

Go to browser and enter http url with the ip of GCP VM. It should render the Streamlit Dashboard with multiple reports.